In [43]:
import pandas as pd
import geopandas as gpd
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')

atl_df = pd.read_csv('unshifted_gt3r_full.csv')

b1 = 0
b2 = b1 + 4
chirop04 = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')

x_min = min(chirop04['x'])
x_max = max(chirop04['x'])
y_min = min(chirop04['y'])
y_max = max(chirop04['y'])

print(f'{x_min} {x_max} {y_min} {y_max}')
# box = np.array([[x_min, y_min], [x_min, y_max], [x_max, y_min], [x_max, y_max]])
# np.savetxt('box.txt', box)

atl_df = atl_df[(atl_df['x'] >= x_min) & (atl_df['x'] <= x_max)]
x_min = min(atl_df['x'])
x_max = max(atl_df['x'])
y_min = min(atl_df['y'])
y_max = max(atl_df['y'])

#we are comparing the 1000:-1000 section of the (straight) ATL against the chiroptera in feature matching
#so we do 1/cos(17.9) * 1000 to get the y cutoffs

atl_df = atl_df[(atl_df['y'] >= y_min + 1051) & (atl_df['y'] <= y_max - 1051)]
print(f'{x_min} {x_max} {y_min} {y_max}')
atl_df['x'] = atl_df['x'] + 41.9 - 42
atl_df['y'] = atl_df['y'] + 104 - 129

chirop04.to_csv('chirop_0_4_auto_shift.csv', index=False)
atl_df.to_csv('0_4_df_auto_shift.csv', index=False)


569358.0299999999 571119.9599999998 9492739.940000001 9497243.080000002
569359.6368558452 571118.4770393161 9492815.605490284 9498288.128618168
